## POS tagging using modified Viterbi
#### Using the Full Test Data @ 5% of given corpus
##### Note: The plain vanilla Viterbi algorithm takes 10-12 min to run on this data.

In [ ]:
#Importing libraries
import nltk
import random
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

### Define helper functions here

In [ ]:
def check_accuracy(tag_seq, test_seq):
    check = [i for i, j in zip(tag_seq, test_seq) if i == j]  
    return(len(check)/len(tag_seq))

### 1. Data Preparation

In [ ]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [ ]:
# view of first few tagged sentences
print(nltk_data[:20])

In [ ]:
# Splitting into train and test
random.seed(1234)
train_set, validn_set = train_test_split(nltk_data,test_size=0.05)
print(len(train_set))
print(len(validn_set))

In [ ]:
print(train_set[:20])

In [ ]:
print(validn_set)

In [ ]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

In [ ]:
# create a set of unique tags from the training data 
tag_set = set([pair[1] for pair in train_tagged_words])

# add a custom unknown tag UNK to cover situations where probability is zero.
#tag_set.add('UNK')
print(tag_set)


In [ ]:
# collect all occurrences of tags in training data
train_tags = [pair[1] for pair in train_tagged_words]


In [ ]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
len(tokens)
V = set(tokens)

vocab_count = len(V)
vocab_count


### 2. Build the vanilla Viterbi based POS tagger

#### 2.1 Define Function to Calculate Emission Probability  

In [ ]:
# for a given word & tag, determine the emission probability based on train set
def word_emission_prob(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag/count_tag)

In [ ]:
# test examples

word='offer'
print("emission probability of the word", word,"given a few tags below:")
t1=time.time()
print("ADJ:", word_emission_prob(word, 'ADJ'))
t2=time.time()
print(t2-t1,'secs.')

t1=time.time()
print("VERB:", word_emission_prob(word, 'VERB'))
t2=time.time()
print(t2-t1,'secs.')

t1=time.time()
print("NOUN:", word_emission_prob(word, 'NOUN'))
t2=time.time()
print(t2-t1,'secs.')

print("PRON:", word_emission_prob(word, 'PRON'))


word='win'
print("emission probability of the word", word,"given a few tags below:")
print("ADJ:", word_emission_prob(word, 'ADJ'))
print("VERB:", word_emission_prob(word, 'VERB'))
print("NOUN:", word_emission_prob(word, 'NOUN'))
print("PRON:", word_emission_prob(word, 'PRON'))



#### 2.2 Set up Transition Probabilities (of tags)

In [ ]:
# define function to compute transition probability of tag1 to tag2 

def transition_prob(t1, t2, tags=train_tags):
    count_t1 = len([t for t in tags if t==t1])
    count_t1_to_t2 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t1_to_t2 += 1
    return (count_t1_to_t2/count_t1)

In [ ]:
# calculate all possible transition probabilities

# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tag_set), len(tag_set)), dtype='float32')
for i, t1 in enumerate(list(tag_set)):
    for j, t2 in enumerate(list(tag_set)): 
        tags_matrix[i, j] = transition_prob(t1,t2)
#tags_matrix

In [ ]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tag_set), index=list(tag_set))
tags_df

In [ ]:
# test with examples

t1="ADJ"
t2="NOUN"
print("Transition probability of", t1, "to", t2, ":", tags_df.loc[t1,t2])

t1="."
t2="DET"
print("Transition probability of", t1, "to", t2, ":", tags_df.loc[t1,t2])

t1="."
t2="NOUN"
print("Transition probability of", t1, "to", t2, ":", tags_df.loc[t1,t2])

t1="ADV"
t2="VERB"
print("Transition probability of", t1, "to", t2, ":", tags_df.loc[t1,t2])

t1="VERB"
t2="NOUN"
print("Transition probability of", t1, "to", t2, ":", tags_df.loc[t1,t2])

t1="NOUN"
t2="."
print("Transition probability of", t1, "to", t2, ":", tags_df.loc[t1,t2])

t2="NOUN"
t1="NUM"
print("Transition probability of", t1, "to", t2, ":", tags_df.loc[t1,t2])

t1="CONJ"
t2="NOUN"
print("Transition probability of", t1, "to", t2, ":", tags_df.loc[t1,t2])

t2="CONJ"
t1="NOUN"
print("Transition probability of", t1, "to", t2, ":", tags_df.loc[t1,t2])

In [ ]:
# Setup words to test algo 

validn_run_words = [tup[0] for sent in validn_set for tup in sent]
validn_run_base = [tup for sent in validn_set for tup in sent]
len(validn_run_words)

#### 2.3 Define "vanilla" Viterbi Function 

In [ ]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    print('tagging, please wait. this could take several seconds/minutes ...\n')
    state = []
    tot_words = len(words)
    words_tagged = 0
    start_time = time.time()
    
   
    T = list(tag_set)
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
            
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_emission_prob(words[key], tag) 

            state_probability = emission_p * transition_p    
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
        
        
        ## print periodic (every 50 words tagged) status message
        words_tagged += 1
        if (words_tagged % 50 == 0):
            end_time = time.time()
            diff = end_time - start_time
            print(words_tagged, 'words done (', int((words_tagged/tot_words)*100),'%)', int(diff), 'secs ...', end=" ")
            
        
    print('\n\n',words_tagged,'words tagged! thanks for your patience.')
    return list(zip(words, state))



### 3. Run the plain, vanilla Viterbi algorithm

In [ ]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(validn_run_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

##### -----------------------------------------------------------------------
##### Save the tagged sequence & validation run base into files to save time if needed to re-run

In [ ]:
len(tagged_seq)
with open("tagged_seq_full.txt", "w") as output:
    output.write(str(tagged_seq))
    
with open("validn_run_base_full.txt", "w") as output:
    output.write(str(validn_run_base))

In [ ]:
len(validn_run_base)

In [ ]:
len(tagged_seq)

#### Execute this if resuming after running Viterbi algo on validation data set.
##### Load up tagged sequence from a file saved in previous run(s) 

import ast
with open('tagged_seq_full.txt', 'r') as f:
    tagged_seq = ast.literal_eval(f.read())

with open('validn_run_base_full.txt', 'r') as f:
    validn_run_base = ast.literal_eval(f.read())


len(tagged_seq)

len(validn_run_base)

##### =================================================

In [ ]:
# calcuate accuracy of the vanilla viterbi algorithm
vanilla_viterbi_accuracy = check_accuracy(tagged_seq, validn_run_base)
print("Accuracy of Vanilla Viteri algorithm:", vanilla_viterbi_accuracy)

In [ ]:
# collect incorrectly tagged cases
incorrect_tagged_cases = [[tagged_seq[i-1],j] for i, j in enumerate(zip(tagged_seq, validn_run_base)) if j[0]!=j[1]]
print('No of Incorrectly tagged cases:', len(incorrect_tagged_cases))
print(incorrect_tagged_cases[:10])

###  Solve the problem of unknown words

In [ ]:
with open('Test_sentences.txt', 'r') as f:
    test_sents = (f.readlines())


In [ ]:
test_sents
test_sents[1]

In [ ]:
# Run Algo on unknown sentences

## Testing
tagged_seq_new_words = []
for sent in test_sents:
    words = word_tokenize(sent)

    start = time.time()
    tagged_seq_new_words.append(Viterbi(words))
    end = time.time()
    difference = end-start
    
tagged_seq_new_words    

### 4. Modfication Technique I :  Incorporate regex based tagger into Viterbi algo.

In [ ]:
patterns = [
(r'.+-.+ed$', 'ADJ'),           # to handle adjectives such as best-selling,
(r'.*ed$', 'VERB'),               # past tense
(r'.+-.+ing$', 'ADJ'),           # to handle adjectives such as best-selling,
(r'.+ing$', 'VERB'),               # past tense
(r'.*ly$', 'ADV'),                # adverb
(r'.+est$','ADJ'),                # superlatives 
(r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
]
regexp_tagger = nltk.RegexpTagger(patterns)   
         
t0 = nltk.DefaultTagger('NOUN')
regex_tagger = nltk.RegexpTagger(patterns, backoff=t0)



In [ ]:
# Viterbi Heuristic modified to include regexp pattern tagging
#

def ViterbiMod1(words, train_bag = train_tagged_words):
    print('tagging, please wait\n')
    state = []
    tot_words = len(words)
    words_tagged = 0
    start_time = time.time()
    
    T = list(tag_set)
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
            
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_emission_prob(words[key], tag) 

            state_probability = emission_p * transition_p    
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        
        # if emission probability is ZERO, invoke the regexp-tagger with backoff 
        if (pmax == 0):
            state_max = regex_tagger.tag([word])[0][1]
            print('ZERO EMISSION ALT TAG:', state_max, 'for word:', word)
            
        state.append(state_max)
        
        words_tagged += 1
        
        #print periodic progress status: for every 50 words processed.
        if ((words_tagged % 50) == 0):
            end_time = time.time()
            diff = end_time - start_time
            print(words_tagged, 'words done (', int((words_tagged/tot_words)*100),'%)', int(diff), 'secs ...', end=" ")
            

    print('\n\n',words_tagged,'words tagged! thanks for your patience.')
    return list(zip(words, state))


In [ ]:
# Run the Viterbi Mod-1 tagger

start = time.time()
tagged_seq_mod1 = ViterbiMod1(validn_run_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

In [ ]:
# Evaluate accuracy of Viterbi Mod-1 algo
mod1_viterbi_accuracy = check_accuracy(tagged_seq_mod1, validn_run_base)
print("Accuracy of Viterbi Mod-1 tagger:", mod1_viterbi_accuracy)

In [ ]:
# collect incorrectly tagged cases
incorrect_tagged_cases_mod1 = [[tagged_seq_mod1[i-1],j] for i, j in enumerate(zip(tagged_seq_mod1, validn_run_base)) if j[0]!=j[1]]
print('No of Incorrectly tagged cases:', len(incorrect_tagged_cases_mod1))



In [ ]:
incorrect_tagged_cases_mod1[:20]

#### Run the Mod-1  tagger on unknown sentences

In [ ]:
## Testing
tagged_seq_mod1_new_words = []
for sent in test_sents:
    words = word_tokenize(sent)

    start = time.time()
    tagged_seq_mod1_new_words.append(ViterbiMod1(words))
    end = time.time()
    difference = end-start
    
tagged_seq_mod1_new_words    

##### The Viterbi Mod-1 tagger has done a better job with unknown words
##### e.g. words like "Google", "Android" etc. that were tagged as CONJ by the vanilla Viterbi algo are now tagged correctly as NOUN. Also, numbers like 2011 are now tagged correctly as NUM


### 5. Modification Technique II : Add Bigram tagger to Viterbi Mod-1 algo.

In [ ]:
# bigram tagger

## Testing
#tagged_seq_bigram_words = []
#for sent in test_sents:
#    words = word_tokenize(sent)
#
#    start = time.time()
#    tagged_seq_bigram_words.append(bi_tagger.tag(words))
#    end = time.time()
#    difference = end-start
    
    
# Evaluate accuracy of Bigram algo
#tagged_seq_bigram = bi_tagger.tag(validn_run_words)
#bigram_accuracy = check_accuracy(tagged_seq_bigram, validn_run_base)
#print("Accuracy of Bigram tagger:", bigram_accuracy)


In [ ]:
t0 = nltk.DefaultTagger('NOUN')
t1 = nltk.RegexpTagger(patterns, backoff=t0)
t2 = nltk.UnigramTagger(train_set, backoff=t1)
reg_bi_tagger = nltk.BigramTagger(train_set, backoff=t1)

t0 = nltk.DefaultTagger('NOUN')
t1 = nltk.UnigramTagger(train_set, backoff=t0)
bi_tagger = nltk.BigramTagger(train_set, backoff=t1)




{'DET', 'X', 'NOUN', 'ADP', 'VERB', 'PRT', 'NUM', 'ADJ', '.', 'PRON', 'CONJ', 'ADV'}

In [ ]:
word='Name'
print(reg_bi_tagger.tag([word]))
print(bi_tagger.tag([word]))

for tag in tag_set:
    print(tag, ':', word_emission_prob(word,tag))
    

In [ ]:
# Viterbi Heuristic modified to include bigram tagging (on top of regex tagger in Mod-1)
#

def ViterbiMod2(words, train_bag = train_tagged_words):
    t0 = nltk.DefaultTagger('NOUN')
    t1 = nltk.UnigramTagger(train_set, backoff=t0)
    bi_tagger = nltk.BigramTagger(train_set, backoff=t1)
    
    print('tagging, please wait\n')
    state = []
    tot_words = len(words)
    words_tagged = 0
    start_time = time.time()
    
    T = list(tag_set)
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
            
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            #emission_p = word_emission_laplace(words[key], tag) 
            emission_p = word_emission_prob(words[key], tag) 

            state_probability = emission_p * transition_p    
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        
        # if emission probability is ZERO, invoke the bigram-tagger with backoff 
        if (pmax == 0):
            state_max = bi_tagger.tag([word])[0][1]
            print('ZERO EMISSION ALT TAG:', state_max, 'for word:', word)
            
        state.append(state_max)
        
        words_tagged += 1
        
        #print periodic progress status: for every 50 words processed.
        if ((words_tagged % 50) == 0):
            end_time = time.time()
            diff = end_time - start_time
            print(words_tagged, 'words done (', int((words_tagged/tot_words)*100),'%)', int(diff), 'secs ...', end=" ")
            

    print('\n\n',words_tagged,'words tagged! thanks for your patience.')
    return list(zip(words, state))


In [ ]:
# Run the Viterbi Mod-2 tagger

start = time.time()
tagged_seq_mod2 = ViterbiMod2(validn_run_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

In [ ]:
# Evaluate accuracy of Viterbi Mod-2 algo
mod2_viterbi_accuracy = check_accuracy(tagged_seq_mod2, validn_run_base)
print("Accuracy of Viterbi Mod-2 tagger:", mod2_viterbi_accuracy)

In [ ]:
# collect incorrectly tagged cases
incorrect_tagged_cases_mod2 = [[tagged_seq_mod2[i-1],j] for i, j in enumerate(zip(tagged_seq_mod2, validn_run_base)) if j[0]!=j[1]]
print('No of Incorrectly tagged cases:', len(incorrect_tagged_cases_mod2))



In [ ]:
incorrect_tagged_cases_mod2[:20]

In [ ]:
## Testing
tagged_seq_mod2_new_words = []
for sent in test_sents:
    words = word_tokenize(sent)

    start = time.time()
    tagged_seq_mod2_new_words.append(ViterbiMod2(words))
    end = time.time()
    difference = end-start
    
tagged_seq_mod2_new_words    

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [ ]:
print('Vanilla Viterbi Accuracy:', vanilla_viterbi_accuracy)
print('Modified-1 Viterbi Accuracy:', mod1_viterbi_accuracy)
print('Modified-2 Viterbi Accuracy:', mod2_viterbi_accuracy)

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

##### Cases tagged incorrectly by original POS tagger - vanilla viterbi

In [ ]:
# Create a df of all incorrect cases from original POS tagger 
#
orig_incorrect_pairs = [j[0] for i,j in incorrect_tagged_cases]
orig_incorrect_case_words = [pair[0] for pair in orig_incorrect_pairs]
orig_incorrect_case_tags = [pair[1] for pair in orig_incorrect_pairs]
orig_incorrect_case_tags
#in_words = [pair[0] for pair in tagged_seq_mod1]

incorrect_df = pd.DataFrame( columns = ['tag'], index=orig_incorrect_case_words)
incorrect_df.tag = orig_incorrect_case_tags
#incorrect_df

##### Cases tagged correctly by original Mod-1 Viterbi

In [ ]:
# Create a df of all Mod-1 word tag pairs
#
mod1_words = [pair[0] for pair in tagged_seq_mod1]
mod1_tags = [pair[1] for pair in tagged_seq_mod1]
mod1_df = pd.DataFrame( columns = ['tag'], index=mod1_words)
mod1_df.tag = mod1_tags
#mod1_df

##### Cases tagged correctly by original Mod-2 Viterbi

In [ ]:
# Create a df of all Mod-2 word tag pairs
#
mod2_words = [pair[0] for pair in tagged_seq_mod2]
mod2_tags = [pair[1] for pair in tagged_seq_mod2]
mod2_df = pd.DataFrame( columns = ['tag'], index=mod2_words)
mod2_df.tag = mod2_tags
#mod2_df

##### Tags from the mod1 Viterbi that are correctly tagged now v/s vanilla Viterbi

In [ ]:
#word='struggled'
#print('incorrect tag for word', word, ':', incorrect_df.loc[word].tag, 
# 'v/s correct tag in mod-1:', mod1_df.loc[word].tag)

#word='534'
#print('incorrect tag for word', word, ':', incorrect_df.loc[word].tag, 
# 'v/s correct tag in mod-1:', mod1_df.loc[word].tag)


incorrect tag for word *struggled* : DET v/s correct tag in Viterbi Mod-1: VERB

incorrect tag for word *534* : DET v/s correct tag in Viterbi Mod-1: NUM


##### Tags from the mod2 Viterbi that are correctly tagged now v/s vanilla Viterbi

In [ ]:

#word='redemption'
#print('incorrect tag for word', word, ':', incorrect_df.loc[word].tag, 
# 'v/s correct tag in mod-2:', mod2_df.loc[word].tag)

#word='534'
#print('incorrect tag for word', word, ':', incorrect_df.loc[word].tag, 
 'v/s correct tag in mod-2:', mod2_df.loc[word].tag)

#word='examined'
#print('incorrect tag for word', word, ':', incorrect_df.loc[word].tag, 
# 'v/s correct tag in mod-2:', mod2_df.loc[word].tag)

incorrect tag for word *redemption*: NOUN v/s correct tag in mod-2: NOUN

incorrect tag for word *534* : DET v/s correct tag in mod-2: NOUN